# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [2]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [3]:
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "cats_and_dogs"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=CatsAndDogs)

Let's add some data

First some cats

In [4]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [5]:
table.add([dict(d) for d in data])

Let's preview the data

In [6]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [7]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [8]:
table.add([dict(d) for d in data])

In [9]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

Here we pass the vector into `search` and make a chained call to `limit` with 1 as the param. Then we'll need to call `to_df` to execute the query and convert the results to a pandas dataframe. In addition to the data columns, you'll also see a "score" column, which contains the distance score between the query vector and the returned vector. In this case, the score is the square of the Euclidean distance.

In [10]:
table.search([10.5, 10.,]).limit(1).to_df()

<ipython-input-10-0e574d743c1f>:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.,]).limit(1).to_df()


,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

**SOLUTION** Remember that cosine distance is the angle between vectors. So while the query vector [10.5, 10.] is closer in Euclidean distance to Samoyed, the angle is slightly closer to shorthair cats.

In [11]:
table.search([10.5, 10.]).metric("cosine").limit(1).to_df()

<ipython-input-11-c2b006291bac>:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.]).metric("cosine").limit(1).to_df()


,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

**HINT** In LanceDB, for additional filtering, you can add a call to `where` in the call chain and pass in a SQL-like filter string.

In [12]:
table.search([10.5, 10.,]).limit(2).where("species='cat'").to_df()

<ipython-input-12-7b8110233f28>:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search([10.5, 10.,]).limit(2).where("species='cat'").to_df()


,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,190.25


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the `vec_to_table` function to convert that in to an Arrow table, which can then be added to the table.

In [13]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(100_000, 16)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [16]:
query = np.random.randn(16)
table.search(query).limit(10).to_df()

<ipython-input-16-bbf71e89c115>:2: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[0.19031097, -0.17567961, -0.026962388, -0.331...",5.316327
1,"[1.0195308, -0.20336777, -0.50310016, 0.630330...",5.529812
2,"[1.0424598, 0.012112393, -0.039127573, -0.2246...",6.661257
3,"[-0.75994295, 0.6096237, 0.46918923, -1.374175...",6.709506
4,"[-0.051707722, 0.5144509, 0.19624312, -1.18210...",7.176255
5,"[1.0304171, 0.7882498, 0.6089045, -0.45904398,...",7.381069
6,"[0.10064966, 0.80228746, 0.07943164, 0.1772442...",7.769833
7,"[0.075128734, -1.4865632, 1.3114115, -0.583752...",7.901943
8,"[0.439243, -0.08010788, -0.57993007, 0.1581364...",8.025385
9,"[0.07216061, -0.56213665, -0.21354531, 0.22674...",8.106874


Please write code to compute the average latency of this query

**SOLUTION** There are several possible solutions. Given that we're in a notebook environment, the easiest is probably using the %timeit magic function to run the command a bunch of times and compute the average

In [17]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

13 ms ± 2.87 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT**
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector

In [18]:
table.create_index(num_partitions=16, num_sub_vectors=8)

Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [19]:
table.search(query).limit(10).to_df()

<ipython-input-19-fe375a661b20>:1: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  table.search(query).limit(10).to_df()


,vector,_distance
0,"[0.19031097, -0.17567961, -0.026962388, -0.331...",4.976735
1,"[1.0195308, -0.20336777, -0.50310016, 0.630330...",5.948328
2,"[1.0424598, 0.012112393, -0.039127573, -0.2246...",6.405713
3,"[-0.051707722, 0.5144509, 0.19624312, -1.18210...",7.176902
4,"[-0.75994295, 0.6096237, 0.46918923, -1.374175...",7.489092
5,"[0.075128734, -1.4865632, 1.3114115, -0.583752...",7.687846
6,"[1.0304171, 0.7882498, 0.6089045, -0.45904398,...",7.956382
7,"[0.10064966, 0.80228746, 0.07943164, 0.1772442...",7.969944
8,"[0.44669387, -1.1939486, -0.57223, -1.2217137,...",8.034195
9,"[0.07216061, -0.56213665, -0.21354531, 0.22674...",8.120974


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [20]:
%timeit table.search(np.random.randn(16)).limit(10).to_arrow();

5.24 ms ± 683 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [21]:
table = db["cats_and_dogs"]

In [22]:
len(table)

4

Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [23]:
table.delete("species='cat'")

In [24]:
len(table)

2

## What if I messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [25]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 19, 55, 969962),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 20, 17, 600844),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 20, 33, 617818),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 34, 18, 426642),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [26]:
table = db["cats_and_dogs"]

In [27]:
len(table)

2

In [28]:
table.restore(3)

In [29]:
table.delete("species='dog'")

In [30]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 19, 55, 969962),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 20, 17, 600844),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 20, 33, 617818),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 34, 18, 426642),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 35, 3, 167224),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2024, 1, 14, 11, 35, 4, 370056),
  'metadata': {}}]

In [31]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [32]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [33]:
db.drop_table("cats_and_dogs")

How would you verify that the table has indeed been deleted?

In [34]:
table.name in db

False